<a href="https://colab.research.google.com/github/lalitapatil/Blockchain-Data-Storage/blob/main/FedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install torch torchvision torchaudio    #https://chatgpt.com/share/68c4ffbf-6198-8001-8b75-9b74fbbc5862
!pip install flwr          # Federated Learning framework
!pip install web3          # Blockchain interaction
!pip install ipfshttpclient # IPFS client
!pip install cryptography   # Encryption for secure storage
#https://chatgpt.com/share/68c54fba-5b98-8001-be47-70e2c01636f8

In [15]:
#get_parameters: Sends the brain’s current knowledge (weights) to the server.
#set_parameters: Updates the brain with knowledge from the server.
#fit: Learns (trains) locally on its own data and then returns new knowledge.
#evaluate: (Here it just returns 0.0, but normally it would test accuracy).

import flwr as fl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Simple model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# Train function
def train(model, train_loader, epochs=1):
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    loss_fn = nn.CrossEntropyLoss()
    for _ in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()

# Flower client
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader):
        self.model = model
        self.train_loader = train_loader

    def get_parameters(self, config):
        return [val.cpu().numpy() for val in self.model.state_dict().values()]

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        for key, val in zip(state_dict.keys(), parameters):
            state_dict[key] = torch.tensor(val)
        self.model.load_state_dict(state_dict)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        train(self.model, self.train_loader)
        return self.get_parameters(config={}), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        return 0.0, len(self.train_loader.dataset), {}


In [17]:
!pip install -U "flwr[simulation]"

In [26]:
import ipfshttpclient

# Replace with your Infura project credentials
client = ipfshttpclient.connect("/dns/infura-ipfs.io/tcp/5001/https")

res = client.add("test.txt")
print(res)


StatusError: HTTPError: 401 Client Error: Unauthorized for url: https://infura-ipfs.io:5001/api/v0/version?stream-channels=true

In [25]:
import ipfshttpclient

# Default local API address
client = ipfshttpclient.connect("/ip4/127.0.0.1/tcp/5001")


# Example: add file
res = client.add("test.txt")
print(res)


ConnectionError: ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5001): Max retries exceeded with url: /api/v0/version?stream-channels=true (Caused by NewConnectionError('<ipfshttpclient.requests_wrapper.HTTPConnection object at 0x781eeb233470>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [23]:
fl.simulation.start_simulation(
    client_fn=lambda cid: FLClient(Net(), train_loader),
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=3)
)

ImportError: Unable to import module `ray`.

To install the necessary dependencies, install `flwr` with the `simulation` extra:

    pip install -U "flwr[simulation]"


In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the MNIST training dataset
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)

# Create a DataLoader for the training data
# Note: In a real federated learning scenario, each client would have its own DataLoader
# with a subset of the data. For this simulation setup, we'll use the full dataset
# for the train_loader passed to the client_fn, but you would typically modify this
# based on how you partition your data for simulation.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print("train_loader defined successfully.")

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 496kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.43MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.16MB/s]

train_loader defined successfully.


In [ ]:
import ipfshttpclient, pickle

# Connect to local/remote IPFS node
client = ipfshttpclient.connect("/dns/localhost/tcp/5001/http")

# Save model parameters
params = fl.common.weights_to_ndarrays(Net().state_dict().values())
with open("model.pkl", "wb") as f:
    pickle.dump(params, f)

# Upload to IPFS
cid = client.add("model.pkl")["Hash"]
print("Stored on IPFS, CID:", cid)


/usr/local/lib/python3.12/dist-packages/urllib3/poolmanager.py:331: DeprecationWarning: The 'strict' parameter is no longer needed on Python 3+. This will raise an error in urllib3 v2.1.0.
  warnings.warn(


ConnectionError: ConnectionError: HTTPConnectionPool(host='localhost', port=5001): Max retries exceeded with url: /api/v0/version?stream-channels=true (Caused by NewConnectionError('<ipfshttpclient.requests_wrapper.HTTPConnection object at 0x7cd5f828df40>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
pragma solidity ^0.8.0;

contract FLStorage {
    mapping(uint => string) public roundCID;

    function storeCID(uint round, string memory cid) public {
        roundCID[round] = cid;
    }

    function getCID(uint round) public view returns (string memory) {
        return roundCID[round];
    }
}


SyntaxError: invalid syntax (ipython-input-3591183005.py, line 1)

In [ ]:
from web3 import Web3

# Connect Ganache (must be running locally or remotely)
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
account = w3.eth.accounts[0]

# ABI + contract address after deployment
abi = [...]
contract_address = "0xYourContractAddress"

contract = w3.eth.contract(address=contract_address, abi=abi)

# Store CID
tx = contract.functions.storeCID(1, cid).build_transaction({
    "from": account,
    "gas": 300000,
    "nonce": w3.eth.get_transaction_count(account),
})
signed_tx = w3.eth.account.sign_transaction(tx, private_key="your_private_key")
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
print("CID stored on blockchain:", cid)


/usr/local/lib/python3.12/dist-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=7545): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7cd5f81ed100>: Failed to establish a new connection: [Errno 111] Connection refused'))